In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import wandb
import seaborn as sns

In [ ]:

def wandb2pd(exp_runs):
    df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
    summary_df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
    config_df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
    name_df = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

    summary = [] 
    config = [] 
    name = [] 
    for exp in exp_runs: 
        summary.append(exp.summary._json_dict) 
        config.append({k:v for k,v in exp.config.items() if not k.startswith('_')}) 
        name.append(exp.name)       

    summary_df = pd.DataFrame.from_records(summary) 
    config_df = pd.DataFrame.from_records(config) 
    name_df = pd.DataFrame({'name': name}) 
    df = pd.concat([name_df, config_df, summary_df], axis=1)
    return df

In [ ]:
colorlist = ["#ff7f00","#984ea3", "y", "k", "w"]

In [ ]:
def create_violinplot_background():

    optimizer_list = ['momentum_sgd', 'adam']

    path_list = [
        f'entity_name/project_momentum_sgd',
        f'entity_name/project_adam',
    ]

    violin_plot_list = []
    
    for i in range(len(path_list)):

        p = path_list[i]
        o = optimizer_list[i]

        api = wandb.Api()
        exp_runs = api.runs(
            path=p,
            filters={'state':'finished'}
            )

        exp_runs
        df = wandb2pd(exp_runs)

        df['optimizer'] = o

        violin_plot_list.append(df)

    df_violinplot = pd.concat(violin_plot_list)
    print(df_violinplot.columns)

    sns.color_palette(colorlist,2)
    sns.set_palette(colorlist)

    ax = sns.violinplot(x='optimizer', y='acc_mixed_rand', data=df_violinplot)
    ax.set_xticklabels(['Momentum', 'Adam'])

    plt.ylim(0, 1.)

    plt.title(f"Background Challenge")

    # plt.legend(bbox_to_anchor=(1.05, 0), borderaxespad=0, loc='lower left', fontsize=10).get_frame().set_linewidth(0)
    plt.grid(linewidth=1)
    plt.savefig(f'figs/violin/violinplot_acc_mixed_rand_background.pdf')
    plt.show()

In [ ]:
create_violinplot_background()